# Iguazio Platform management API Introduction

- [Defining some variables](#defining-variables)
- [Opening a control session](#opening-control-session)
- [Example - Get you own user's data](#example-get-self)
- [Example - Get data containers](#example-get-containers)
- [Example - Create 2 example nuclio functions](#example-create-functions)
- [Example - Get nuclio functions](#example-get-functions)
- [Example - Create API Gateway](#example-create-apigateway)
- [Example - Get API Gateways](#example-get-apigateways)
- [Example - Invoke your functions via API Gateways](#example-invoke-via-apigateways)
- [Cleanup - Remove example leftovers](#cleanup)

<a id="defining-variables"></a>
## Defining some variables

In [1]:
# Substitute your username and password here:

IGUAZIO_USERNAME = 'admin'
IGUAZIO_PASSWORD = 'silvester88'
DASHBOARD_URL = 'https://datanode-dashboard'

<a id="opening-control-session"></a>
## Opening a Control Session

In [2]:
import urllib3
import requests
import json
import time
import os

# HACK for systems without production cert - shut up the no cert verification WARN
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

session = requests.Session()
session.auth = (IGUAZIO_USERNAME, IGUAZIO_PASSWORD)
auth = session.post(f'{DASHBOARD_URL}/api/sessions', verify=False)

session_id = auth.json()['data']['id']
print(f'Session opened, your session ID is: {session_id}')

Session opened, your session ID is: 5ad62e68-4c64-49dc-869b-4a6f2e126caf


<a id="example-get-self"></a>
## Example - Get your own User's data

In [3]:
response = session.get(f'{DASHBOARD_URL}/api/self')
print(json.dumps(response.json(), indent=2))

{
  "included": [],
  "meta": {
    "ctx": "11509044878258052005"
  },
  "data": {
    "attributes": {
      "username": "admin",
      "first_name": "John",
      "last_name": "Doe",
      "uid": 50,
      "password_changed_at": "2020-05-04T07:32:54.223000+00:00",
      "created_at": "2020-05-04T07:32:54.218000+00:00",
      "send_password_on_creation": false,
      "assigned_policies": [
        "Data",
        "Security Admin",
        "Application Admin",
        "Function Admin",
        "Service Admin"
      ],
      "updated_at": "2020-05-12T22:37:04.161000+00:00",
      "admin_status": "up",
      "data_access_mode": "enabled",
      "authentication_scheme": "local",
      "operational_status": "up",
      "email": "admin-default-tenant@iguazio.com"
    },
    "type": "user",
    "id": "dce476fc-3ec4-4103-a059-27c5ba008baa"
  }
}


<a id="example-get-containers"></a>
## Example - Get data containers

In [4]:
response = session.get(f'{DASHBOARD_URL}/api/containers')
print(json.dumps(response.json(), indent=2))

{
  "included": [],
  "meta": {
    "ctx": "12435627524985450772"
  },
  "data": [
    {
      "attributes": {
        "name": "users",
        "imports": [],
        "created_at": "2020-05-04T07:28:07.048000+00:00",
        "operational_status": "up",
        "updated_at": "2020-05-04T07:28:07.971000+00:00",
        "id": 1026,
        "admin_status": "up",
        "data_lifecycle_layers_order": [],
        "data_policy_layers_order": [
          "2b2bd932-51f9-4f07-8011-ab0f74a116d2",
          "7e1dc942-13d5-4d76-8b7a-738e83ee6f91"
        ],
        "properties": [
          "system_provided"
        ]
      },
      "type": "container",
      "id": 1026
    },
    {
      "attributes": {
        "name": "bigdata",
        "imports": [],
        "created_at": "2020-05-04T07:28:07.041000+00:00",
        "operational_status": "up",
        "updated_at": "2020-05-04T07:28:08.484000+00:00",
        "id": 1027,
        "admin_status": "up",
        "data_lifecycle_layers_order": [],
   

<a id="example-create-functions"></a>
## Example - Create 2 example nuclio functions

In [263]:
example_function_names = ['hello-world-example-1', 'hello-world-example-2']

for function_name in example_function_names:
    request = {
        'data': {
            'type': 'function',
            'id': function_name,
            'attributes': {
                'spec': {
                    'description': 'Showcases unstructured logging and a structured response.',
                    'handler': 'helloworld:handler',
                    'build': {
                        'function_source_code': 'IyBDb3B5cmlnaHQgMjAxNyBUaGUgTnVjbGlvIEF1dGhvcnMuCiMKIyBMaWNlbnNlZCB1bmRlciB0aGUgQXBhY2hlIExpY2Vuc2UsIFZlcnNpb24gMi4wICh0aGUgIkxpY2Vuc2UiKTsKIyB5b3UgbWF5IG5vdCB1c2UgdGhpcyBmaWxlIGV4Y2VwdCBpbiBjb21wbGlhbmNlIHdpdGggdGhlIExpY2Vuc2UuCiMgWW91IG1heSBvYnRhaW4gYSBjb3B5IG9mIHRoZSBMaWNlbnNlIGF0CiMKIyAgICBodHRwOi8vd3d3LmFwYWNoZS5vcmcvbGljZW5zZXMvTElDRU5TRS0yLjAKIwojIFVubGVzcyByZXF1aXJlZCBieSBhcHBsaWNhYmxlIGxhdyBvciBhZ3JlZWQgdG8gaW4gd3JpdGluZywgc29mdHdhcmUKIyBkaXN0cmlidXRlZCB1bmRlciB0aGUgTGljZW5zZSBpcyBkaXN0cmlidXRlZCBvbiBhbiAiQVMgSVMiIEJBU0lTLAojIFdJVEhPVVQgV0FSUkFOVElFUyBPUiBDT05ESVRJT05TIE9GIEFOWSBLSU5ELCBlaXRoZXIgZXhwcmVzcyBvciBpbXBsaWVkLgojIFNlZSB0aGUgTGljZW5zZSBmb3IgdGhlIHNwZWNpZmljIGxhbmd1YWdlIGdvdmVybmluZyBwZXJtaXNzaW9ucyBhbmQKIyBsaW1pdGF0aW9ucyB1bmRlciB0aGUgTGljZW5zZS4KCgpkZWYgaGFuZGxlcihjb250ZXh0LCBldmVudCk6CiAgICBjb250ZXh0LmxvZ2dlci5pbmZvKCdUaGlzIGlzIGFuIHVuc3RydWN0dXJlZCBsb2cnKQoKICAgIHJldHVybiBjb250ZXh0LlJlc3BvbnNlKGJvZHk9J0hlbGxvLCBmcm9tIG51Y2xpbyA6XScsCiAgICAgICAgICAgICAgICAgICAgICAgICAgICBoZWFkZXJzPXt9LAogICAgICAgICAgICAgICAgICAgICAgICAgICAgY29udGVudF90eXBlPSd0ZXh0L3BsYWluJywKICAgICAgICAgICAgICAgICAgICAgICAgICAgIHN0YXR1c19jb2RlPTIwMCkK',
                        'code_entry_type': 'sourceCode'
                    },
                    'runtime': 'python:3.6',
                    'replicas': 1,
                },
            },
        },
    }

    response = session.post(f'{DASHBOARD_URL}/api/functions', json=request)
    if response.status_code == 409:
        print(f'Function {function_name} already exists (That\'s OK!)')
    elif not response.ok:
        print(f'OOPS! Failed to create {function_name}: {response.text}')
    else:
        print(json.dumps(response.json(), indent=2))
    
print('Done creating functions!')

Function hello-world-example-1 already exists (That's OK!)
Function hello-world-example-2 already exists (That's OK!)
Done!


In [265]:
# Wait for the functions to be ready

for function_name in example_function_names:
    state = 'unknown'
    while True:
        response = session.get(f'{DASHBOARD_URL}/api/functions/{function_name}')
        
        if not response.ok:
            raise RuntimeError(f'Unexpected response code: {response.status_code}')

        response_json = response.json()
        
        # uncomment this to inspect the response
        # print(json.dumps(response_json, indent=2)) 
                
        state = response_json['data']['attributes']['status']['state']
        if state in ['ready', 'error']:
            break
        else:
            print(f'Function {function_name} is in state {state}...')
            time.sleep(3)
    if state == 'ready':
        print(f'Function {function_name} is ready!')
    else:
        print(f'OOPS! Something went wrong and {function_name} is not ready!')

print('Done waiting for functions readiness!')

Function hello-world-example-1 is ready!
Function hello-world-example-2 is ready!
Done waiting for functions readiness!


<a id="example-get-functions"></a>
## Example - Get nuclio functions

In [266]:
response = session.get(f'{DASHBOARD_URL}/api/functions')

response_json = response.json()
retrieved_function_names = [func['attributes']['metadata']['name'] for func in response_json['data']]
print(f'Retreived functions: {retrieved_function_names}')

# NOTE: uncomment to see full response json
# print(json.dumps(response_json, indent=2))

Retreived functions: ['hello-world-example-1', 'hello-world-example-2']


<a id="example-create-apigateway"></a>
## Example - Create API Gateway for the example nuclio functions

In [267]:
api_gateway_name = 'hello-world-gateway'
namespace_domain = os.getenv('IGZ_NAMESPACE_DOMAIN')
api_gateway_host = f'{api_gateway_name}.default.{namespace_domain}'

request = {
   'data':{
      'attributes':{
         'meta':{
            'name':'example-gateway'
         },
         'spec':{
            'authentication_mode':'none',
            'host': api_gateway_host,
            'upstreams':[
               {
                  'kind':'nucliofunction',
                  'nucliofunction':{
                     'name': example_function_names[0],
                  }
               },
               {
                  'kind':'nucliofunction',
                  'nucliofunction':{
                     'name': example_function_names[1],
                  },
                  'percentage':5
               },
            ],
            'description':'My example gateway',
            'name': api_gateway_name,
         },
      },
      'type':'api_gateway',
   },
}


response = session.post(f'{DASHBOARD_URL}/api/api_gateways', json=request)
if response.status_code == 409:
    print(f'API Gateway with name {api_gateway_name} already exists (That\'s OK!)')
else:
    print(json.dumps(response.json(), indent=2))

print('Done creating API gateway!')

API Gateway with name hello-world-gateway already exists (That's OK!)


In [269]:
# Wait for the api gateway to be ready

state = 'unknown'
while True:
    response = session.get(f'{DASHBOARD_URL}/api/api_gateways/{api_gateway_name}')
    
    # keep trying on 404, it might take a bit for the api gateway to be created
    if response.status_code == 404:
        time.sleep(3)
        continue
        
    if not response.ok:
        raise RuntimeError(f'Unexpected response code: {response.status_code}')

    response_json = response.json()

    # uncomment this to inspect the response
    # print(json.dumps(response_json, indent=2)) 

    state = response_json['data']['attributes']['status']['state']
    if state in ['ready', 'error']:
        break
    else:
        print(f'API Gateway {api_gateway_name} is in state {state}...')
        time.sleep(3)

if state == 'ready':
    print(f'API Gateway {api_gateway_name} is ready!')
else:
    print(f'OOPS! Something went wrong and {api_gateway_name} is not ready!')

print('Done waiting for API gateway readiness!')

API Gateway hello-world-gateway is ready!
Done waiting for API gateway readiness!


<a id="example-get-apigateways"></a>
## Example - GET API Gateways

In [3]:
response = session.get(f'{DASHBOARD_URL}/api/api_gateways')

response_json = response.json()

# NOTE: uncomment to see full response json
# print(json.dumps(response_json, indent=2))

api_gateway_names = [gateway['attributes']['status']['name'] for gateway in response_json['data']]
print(f'Retreived API gateways: {api_gateway_names}')

Retreived API gateways: ['myapi']


In [9]:
r = session.get(f'{DASHBOARD_URL}/api/api_gateways/myapi')
r.json()['data']['attributes']['spec']

{'path': '/',
 'host': 'myapi.sk-project.default-tenant.app.yh55.iguazio-cd2.com',
 'upstreams': [{'extra_annotations': {'entries': []},
   'kind': 'nucliofunction',
   'nucliofunction': {'name': 'sk-project-sklearn-server-v1'},
   'percentage': 0},
  {'extra_annotations': {'entries': []},
   'kind': 'nucliofunction',
   'nucliofunction': {'name': 'sk-project-sklearn-server-v2'},
   'percentage': 46}],
 'name': 'myapi',
 'authentication_mode': 'none'}

<a id="example-invoke-via-apigateways"></a>
## Example - Invoke your functions via API Gateways

In [271]:
# Retrieve the API gateway we previous created
response = session.get(f'{DASHBOARD_URL}/api/api_gateways/{api_gateway_name}')

response_json = response.json()

# NOTE: uncomment to see full response json
# print(json.dumps(response_json, indent=2))

# extract the host (which we previously provided when creating the api gateways)
api_gateway_host = response_json['data']['attributes']['spec']['host']

# launch an example request
api_gateway_response = requests.get(f'http://{api_gateway_host}')

# print the response from your newly created api gateway
print(F'API Gateway response ({api_gateway_response.status_code}): {api_gateway_response.text}')

API Gateway response (200): Hello, from nuclio :]


<a id="cleanup"></a>
## Cleanup - Remove example leftovers

In [275]:
print(f'Removing {api_gateway_name}')
response = session.delete(f'{DASHBOARD_URL}/api/api_gateways/{api_gateway_name}')
print(response.status_code, response.text)

for function_name in example_function_names:
    print(f'Removing {function_name}')
    response = session.delete(f'{DASHBOARD_URL}/api/functions/{example_function_names[0]}')
    if response.ok:
        print(response.status_code, response.text)
    else:
        print(f'Function {function_name} already removed')

print('Done cleaning up! GOOD BYE :]')

Removing hello-world-gateway
204 
Removing hello-world-example-1
Function hello-world-example-1 already removed
Removing hello-world-example-2
Function hello-world-example-2 already removed
Done cleaning up! GOOD BYE :]
